In [38]:
import requests
import lxml
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm, trange
import os
import pdfplumber


def read_pdf(file_name):
    with pdfplumber.open(f"./pdf_hwp/{file_name}") as pdf:
        pages = pdf.pages
        text = ""
        for page in pages:
            text += page.extract_text()
        if len(text) < 10:
            text = f"{file_name} 한글 인식 불가 ! OCR 필요"
    return text


# file_id = '1138672129786_104849.pdf'
# file_name = 'down.pdf'
def download_file(file_id, file_name=None):
    if not file_name:
        return
    download_url = f"https://file.scourt.go.kr//AttachDownload?&name=downForm&file={file_id}&path=003&downFile={file_name}"

    with open(f"./pdf_hwp/{file_name}", "wb") as file:
        response = requests.get(download_url)
        file.write(response.content)


def make_csv_file(body_content, file_name=None):
    if not file_name:
        return
    file_name = re.sub(r".pdf", r".csv", file_name)
    pd_body_content = pd.DataFrame(body_content)
    pd_body_content.to_csv(path_or_buf=f"./pdf_hwp/{file_name}", index=False)


for i in tqdm(range(1, 1000)):
    base_url = "https://busan.scourt.go.kr/dcboard/new/DcNewsViewAction.work?&gubun=44&cbub_code=000410&searchWord=&pageIndex=1"
    params = {"seqnum": i}
    resp = requests.get(base_url, params=params)
    soup = BeautifulSoup(resp.content, "lxml")
    tbody_list = []
    tbody = soup.find("tbody")
    title = tbody.find("td", class_="title").text
    court = tbody.select_one("tr:nth-child(2) > td:nth-child(2)").text
    downloads = tbody.select("tr:nth-child(3) > td > a")
    url_regex = re.compile(r"\'(.*\.pdf)\'\,\'(.*.pdf)\'")
    body = tbody.find("div", class_="view_content").getText()
    file_list = []
    for download in downloads:
        file_id = url_regex.search(str(download)).group(1)
        file_name = url_regex.search(str(download)).group(2)
        file_list.append(file_name)

        download_file(file_id, file_name)

    body_content = [
        {
            "제목": title,
            "법원": court,
            "내용": body,
            "file_name": file_list,
            "file_body": None,
        }
    ]
    text_list = []
    for file_name in file_list:
        text_list.append(read_pdf(file_name))

    body_content[0]["file_body"] = text_list

    make_csv_file(body_content, file_name)


 10%|█         | 1/10 [00:00<00:08,  1.01it/s]

['전  주  지  방  법  원제 1  형 사 부판         결사       건 2017노1590   가. 사기나. 위계공무집행방해다. 업무방해피  고  인 1.가.나.다. A (87-1),  강사2.가.나.다. B (63-1),  공예가항  소  인 쌍방검       사 김영기(기소), 최영준(공판)원 심 판 결 전주지방법원 남원지원 2017. 10. 17. 선고 2017고단42, 2017고정14(병합) 판결판 결 선 고 2018. 6. 1.  주       문피고인들과 검사의 항소를 모두 기각한다.  이       유1. 항소이유의 요지- 1 -  가. 피고인들    (1) 사실오인 및 법리오해      (가) 전통공예인 옻칠공예는 협업을 전제로 하여 계승, 발전되어 왔고 나전(자개)은 장식으로 대체가능한 것에 불과함에도 원심판결은 제45회 대한민국공예품대전 및 그 지역예선인 제38회 전라북도공예품대전(이하 위 각 대전을 통칭하여 ‘이 사건 각 대전’이라 한다)에서 요구하는 ‘직접 제작’의 의미를 그릇 해석하고 그에 관한 사실을 오인한 잘못이 있다.       (나) 나전작업을 외부작업자에게 의뢰하는 것이 옻칠공예의 작업방식이었으므로 피고인 A에게 이 사건 각 범행의 고의 내지 위법성 인식이 있었다고 볼 수 없고, 피고인 A이 이 사건 각 대전에 향의 여운을 출품하면서 C를 협력자로 기재하지 아니한 것과 현장실사 과정에서 나전작업을 재연한 것이 기망행위 내지 위계행위로 볼 수 없다. 나아가 피고인 A이 위와 같이 C를 협력자 또는 공동제작자로 밝히지 않은 행위와 이 사건 각 대전에서의 수상이라는 결과 사이에 인과관계도 인정되지 않는다.       (다) 피고인 B에게 공동정범으로서의 공동가공의사 내지 공동가공행위가 있었다고 볼 수 없다.     (2) 양형부당      피고인들에 대한 원심의 형(각 벌금 500만 원)은 너무 무거워서 부당하다.  나. 검사    (1) 사실오인 및 법리오해      모방품 여부를 판단함에 있어서는 디자인보호법 제33

 20%|██        | 2/10 [00:02<00:08,  1.05s/it]

['전  주  지  방  법  원제 1  형 사 부판         결사       건 2017노1766  의료법위반피  고  인 A (60-1),  의사항  소  인 검사검       사 윤소현(기소), 최영준(공판)원 심 판 결 전주지방법원 2017. 11. 30. 선고 2017고정503 판결판 결 선 고 2018. 6. 22.  주       문원심판결을 파기한다.피고인을 벌금 1,000,000원에 처한다.피고인이 위 벌금을 납입하지 아니하는 경우 100,000원을 1일로 환산한 기간 피고인을 노역장에 유치한다. 위 벌금에 상당한 금액의 가납을 명한다.  이       유1. 항소이유의 요지(법리오해)- 1 -  의료법 제3조 제1항이 정하고 있는 ‘의료업’의 정의에 따르면 대가의 취득 여부가 의료업의 요건에 해당한다고 보기 어렵고, 업무의 계속, 반복성에 의하여 의료업에 해당하는지 여부가 판단되어야 할 것이다. 피고인은 2014년 7월경부터 2014년 10월 31일까지 매주 수요일과 토요일 정기적으로 B가 개설한 ◯◯안과에서 스마일 안과수술을 시행하였으므로, 피고인은 자신이 개설한 의료기관이 아닌 B가 서울시에 개설한 ◯◯안과에서 의료업을 영위한 것이다. 그럼에도 이 사건 공소사실을 무죄로 판단한 원심판결에는 법리오해의 잘못이 있다.2. 판단  가. 공소사실의 요지    피고인은 전주시에서  ◊◊안과의원을 개설한 의사이다.    의료인은 이 법에 따른 의료기관을 개설하지 아니하고는 의료업을 할 수 없으며, 특별한 경우 외에는 그 의료기관 내에서 의료업을 하여야 한다.    그럼에도 불구하고 피고인은 2014. 7. 5.경부터 같은 해 10. 31.경까지 서울시에 소재한 의사 B가 개설한 의료기관인 ‘◯◯안과의원’에서 매주 수요일과 토요일 정기적으로 방문하여 환자 58명의 안과 수술을 하는 방법으로 의료업을 하였다.  나. 원심의 판단    원심은 의료법은 의료행위와 의료업을 구분하고 있는 점, 업은 직업과 같은 말로서 생계를 유지하기 위해 자신의 적성과

 30%|███       | 3/10 [00:02<00:06,  1.06it/s]

...

['창 원 지 방 법 원판 결사 건 2018고단197 사기, 사기미수2018고단699(병합) 사기, 사기미수2018고단799(병합) 야간건조물침입절도, 절도피 고 인 A검 사 김희영, 황보영, 이세종(기소), 이준석(공판)변 호 인 변호사 X(국선)판 결 선 고 2018. 5. 16.주 문피고인을 징역 1년 6월에 처한다.이 유범 죄 사 실범죄전력○ 사건 : 대구고등법원 2015. 6. 4. 선고 강도치상, 상습절도 등○ 판결 : 징역 3년 6월○ 경과 : 2017. 5. 30. 천안개방교도소 가석방 ▶ 2017. 11. 11. 가석방 기간 경과- 1 -범죄사실1.『2018고단197』사건피고인은 2018. 1. 12. 14:50경 김해시 인제로159 앞 사거리를 유토피아 아파트 사거리 방면에서 가야 유치원 방면으로 횡단보도 보행자신호에 우회전하던 피해자 정○경이 운전하는 쏘나타 승용차의 조수석 뒤 문짝 부분에 의도적으로 달려들어 부딪힌 사실이 있었다.그럼에도 피고인은 피해자에게 피고인의 바지를 걷어 올려 왼쪽 다리를 내보이며 “당신의 차가 나를 쳤다. 병원에서 외출을 나온 상태라 보험처리는 곤란하니 여기서 합의를 하자.”고 거짓말하였다.피고인은 위와 같이 피해자를 기망하여 이에 속은 피해자로부터 재물을 교부받으려고하였으나 피해자가 보험회사에 사고 접수할 것을 계속 주장하자 자신의 범행이 발각될것을 우려하여 현장을 이탈함으로써 그 뜻을 이루지 못하고 미수에 그쳤다.피고인은 이를 비롯하여 그때부터 같은 날 20:05경까지 별지 범죄일람표에 기재된 것과같이 위와 같은 방법으로 피해자 3명으로부터 합계 26만 원을 교부받아 편취하고, 피해자 2명으로부터 재물을 편취하려고 하였으나 그 뜻을 이루지 못하고 미수에 그쳤다.2.『2018고단699』사건피고인은 2018. 1. 8. 20:50경 창원시 의창구 창원대로에 있는 용원교차로를 창원터널 쪽에서 명곡광장 쪽으로 우회전하여 횡단보도를 통과하던 피해자 심○○ 운전의 1크루즈 승용차를 발견하고, 위 승용차 조수석 뒷부분을 손

 90%|█████████ | 9/10 [00:06<00:00,  1.56it/s]

['창 원 지 방 법 원 평소 피해자와 자주 통화하는 양○○의 휴대전화가 ‘통화중’이 아니라는 것을 확인하고피해자와 양○○이 함께 있다고 생각하여 창원시 의창구(이하 기재하지 않음)에 있는제 형 사 부양○○의 집에 찾아갔으나 양○○이 비밀번호를 변경하여 문이 열리지 않자 피해자와판 결 양○○이 그곳에 함께 있다고 확신하고 격분하여 피해자를 죽이기로 마음먹었다.이에 피고인은 창원시 의창구 봉곡동에 있는 피고인의 집에 가서 그곳에 있던 부엌사 건 2018고합9 살인미수, 상해칼(전체길이 26cm, 칼날길이 15cm)과 주방용 가위(전체길이 24cm, 날길이 12cm)를 피피 고 인 A고인의 점퍼 왼쪽 안주머니에 넣고, 피고인의 차에 있던 공업용 장갑을 챙긴 후, 같은검 사 김경완(기소), 장일희, 이승필(공판)날 06:05경 다시 위 양○○의 집으로 찾아가 열려있는 뒤쪽 베란다 창문을 통해 안방변 호 인 변호사 B, C, 공익법무관 D, E, F까지 들어가, 그곳에서 양○○과 함께 있던 피해자가 피고인을 보고 놀라 일어서려고판 결 선 고 2018. 4. 17.하자 오른손으로 위 부엌칼을 잡고 피해자에게 “개새끼야, 너 죽인다. 왜 남의 마누라를 탐내.”라고 말하며 피해자의 머리를 세게 내리쳐 찌르고, 이에 피해자가 반항하자주 문피해자를 바닥에 넘어뜨리고 피해자의 배 위에 올라타 재차 위 부엌칼로 피해자의 오피고인을 징역 3년 6월에 처한다.른쪽 가슴 부위를 1회 찔러 피해자를 살해하려고 하였으나, 피해자가 피고인을 제압하압수된 증 제1 내지 3호(가위 1개, 용접용 장갑 1켤레, 식칼 1개)를 각 몰수한다.여 바닥에 눕히고 곧이어 양○○의 신고를 받고 경찰관이 출동하는 바람에 피해자에게약 2주간의 치료가 필요한 두피의 열린 상처(길이 12cm), 흉곽 부분의 열린 상처(길이이 유4.5cm)를 가하는데 그침으로써 미수에 그쳤다.범 죄 사 실2. 상해1. 살인미수피고인은 위 일시경 위 피해자 양○○(여, 61세)의 집에서 피고인의 제1항과 같은 폭피고인은 피해자 김○

100%|██████████| 10/10 [00:07<00:00,  1.40it/s]

['창 원 지 방 법 원판 결사 건 2018고단1061 강제추행, 절도피 고 인 A검 사 김용선(기소), 김다현(공판)변 호 인 변호사 B판 결 선 고 2018. 6. 28.주 문피고인을 징역 1년 6개월에 처한다.다만 이 판결 확정일부터 3년간 위 형의 집행을 유예한다.피고인에게 보호관찰 받을 것과 40시간의 성폭력 치료강의 수강을 명한다.이 유범 죄 사 실1. 강제추행피고인은 술에 취한 피해자 C(여, 26세)과 피해자의 친구가 창원시 성산구 원이대로672(상남동), D빌딩 지하주차장으로 걸어 들어간 후 피해자의 친구만 나오는 모습을확인하고 술에 취해 혼자 있는 피해자를 강제추행하기로 마음먹었다.피고인은 2018. 3. 18. 02:17경 위 성인빌딩 지하주차장에서 피해자가 운전석에 타고- 1 -있던 56거10○○호 승용차로 다가가 운전석 창문을 통해 피해자를 살펴보자, 피해자가피고인을 주차 관리인으로 착각하여 위 승용차를 빼달라는 것으로 생각하고 피고인에게 “왜요”라고 하면서 차문을 열었다. 그러자 피고인은 피해자에게 “너무 예뻐서 그렇다.”라고 말하며 갑자기 위 승용차 안으로 몸을 넣어 오른손으로 그곳에 누워 있던 피해자의 가슴부위를 눌러 피해자가 반항하지 못하게 한 후 피해자가 ‘왜 이러세요’라고말하며 피고인을 밀쳤음에도 이를 무시하고 피해자의 볼에 뽀뽀를 하고, 왼손을 피해자의 치마 속으로 넣어 속바지 위로 음부를 수회 만져 피해자를 강제로 추행하였다.2. 절도피고인은 제1항 기재와 같은 날 02:33경 위 지하주차장에서 피해자 C이 위와 같이피해를 당한 후 남자친구인 E에게 위 피해 내용을 알리기 위해 위 56거10○○호 승용차를 시정하지 않은 채 자리를 비운 틈을 이용하여 위 승용차 전면 유리창에 장착되어있는 피해자 소유인 시가 200,000원 상당의 블랙박스 1개, 위 승용차 조수석에 있던시가 15,000원 상당의 파우치 화장품 1개, 시가 30,000원 상당의 아이폰 충전기 1개,주민등록증 1장, 농협체크카드 1장이 들어있는 피해자 소유인 시

In [18]:


# print(read_pdf('2005허5860.pdf'))
# with open('./pdf_hwp/eee.txt', 'w') as f:
#     text_desc = read_pdf('2005허5860.pdf')
#     f.write(text_desc)
    


2005허5860.pdf 한글 인식 불가 ! OCR 필요
